## RSA LAB!!!  

In this lab we will work through, in a hands on way, real life RSA and Digital Signatures.  

I am not going to give much advice or help in this assignment, but lots of help and advice will be given on Slack.  
I will however give resources to look at for help.

In [ ]:
########## Question 1 (0 points) ##########

## Resoures used / People worked with
# https://agrohacksstuff.io/posts/working-with-bytecode-in-python/
# https://cryptobook.nakov.com/digital-signatures/rsa-sign-verify-examples



######## student answers here ########


Lets work on encoding. 
We want to turn our text "I want to send a secret message" into a large integer. 
once it's an integer we can do the RSA exponentiation math on it to create ciphertext.  
  
There are a couple different formulas for doing this, which I encourage you to look at  
however for this class we will simplify code as much as possible  
use the `bytes_to_long()` function from pycryptodome. 


In [ ]:
########## Question 2 (10 points) ##########
from Crypto.Util.number import bytes_to_long
# install pycrpytodome
# use bytes_to_long to encode the message
message = b'This is a secret message for Aaron and Matt: I want an A!'

encoded_message = (bytes_to_long(message))
print('encoded_message:', encoded_message)
#encoded_message: 61350900581473535129405983010356211867018600038007221145744717313351488407028771212587494491695427024167991689288623317672691990347661601

Back in the days of HW2, we might continue here with some simple math, like:  
`pow(encoded_message, e, N)`. 

But now we will continue instead with a realistic implementation using my public encryption key.  

I created a private key by doing this code:

```python
# create key
from Crypto.PublicKey import RSA
private_key = RSA.generate(2048, e=65537)

# write key to file
with open('private_key.pem', "wb") as f:
    data = private_key.export_key()
    f.write(data)
```

Then I can create a public key:
```python
# create key and write to file
with open('public_key.pem', 'wb') as f:
    data = private_key.public_key().export_key()
    f.write(data)
```

In [ ]:
########## Question 3 (10 points) ##########

# use the documentation and provide me the public modulus and public exponent
# https://pycryptodome.readthedocs.io/en/latest/src/public_key/rsa.html#Crypto.PublicKey.RSA.RsaKey

import Crypto.PublicKey.RSA


n = Crypto.PublicKey.RSA.import_key(open('public_key.pem').read()).n
e = Crypto.PublicKey.RSA.import_key(open('public_key.pem').read()).e
print('n:', n, '\ne:', e)

# n: 22906335060190800334132952890499973331143740797726805958507040016801432104850654831641504824820911475519480713024418251113565374596797253909499605367697368009042784056959268434699394055540499053370687602548424051341234071216620179505349120024318276414412021675413008616635740690540438212012951165403946456109168824490671489505019598024328032354881385974340981944258223245774818256281859869073022548567914622435902591610880524862550427584796695318714761562636141429803958842074373934764212139739847246823168166467638689002401000353035875413869429591952212794820173777686272041550035599870817593246969300817703949738103 
# e: 65537

In [ ]:

########## Question 4 (10 points) ##########

# use the documentation and provide me my private modulus' factorization and private exponent
# https://pycryptodome.readthedocs.io/en/latest/src/public_key/rsa.html#Crypto.PublicKey.RSA.RsaKey
import Crypto.PublicKey.RSA



p = Crypto.PublicKey.RSA.import_key(open('private_key.pem').read()).p
q = Crypto.PublicKey.RSA.import_key(open('private_key.pem').read()).q
d = Crypto.PublicKey.RSA.import_key(open('private_key.pem').read()).d
print('p:', p, '\nq:', q, '\nd:', d)

# p: 151064822135023914000685662310325292385086444733780794869798900493764743886495074073170738073874936276823658115464391669173524243342942522141935728228899721501473106652726180501167711950517139742612453712321653347072442283556886579278167382437989275804733528282568691001432866566172783909104423634844878829829 
# q: 151632489526362319538664915920542644673322390304547808107467309438871201483336069242452553019425073174227286559134582591427842777764538823016433416019927382481426602228162448632742306342315327522800971479202640649807992739017581934752058785919253406486607057012485375856874636869940080138937537212418442181707 
# d: 2473186549367687003743301869984555461665518865445700580777206999998274769579370944484722952537234990932997322510349627613097770582219774610733161535954141569372823595633669277567372817446702951029967582825467271728803153759384842000394439374583458563992545666954887299865945971378978909443576032567836872658594942031503967083464214579986626415925589914669661031827884278890081664281451931938589760683131254123332014796544368098316852351323143843582785796611206435593783884316906511171399633967973458964412050433644894625311862311035673210907765131534217442303645630244903770220160642238307683656737565918959334251937

In [ ]:
########## Question 5 (20 points) ##########

# encrypt the message with RSA
# use PKCS1_OEAP as the cipher (an RSA cipher with OEAP padding)
# https://pycryptodome.readthedocs.io/en/latest/src/examples.html

from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES, PKCS1_OAEP
message = b'This is a secret message for Aaron and Matt: I want an A!'

recipient_key = RSA.import_key(open("public_key.pem").read())
session_key = get_random_bytes(16)

cipher_rsa = PKCS1_OAEP.new(recipient_key)
enc_session_key = cipher_rsa.encrypt(session_key)

cipher_aes = AES.new(session_key, AES.MODE_EAX)
ciphertext, tag = cipher_aes.encrypt_and_digest(message)

with open("encrypted_message.bin", "wb") as f:
    f.write(enc_session_key)
    f.write(cipher_aes.nonce)
    f.write(tag)
    f.write(ciphertext)

print('ciphertext:', ciphertext)
# ciphertext: b'$\xb2[\x03\xac4F\xf7y\xf3\x07j\x16\xeeEb$\xaa`\x11\xac\x85T\xa5s\xd2\xd0\xc4\x05\x16\r\xce\x1aF\x91\x81\xe2`\xeb[t\x91Z\x82\xe2lQ\xa9,#\x13\x11+\xb4\xc7\x96B'

In [ ]:
########## Question 6 (20 points) ##########

# decrypt the message with RSA
# use PKCS1_OEAP as the cipher (an RSA cipher with OEAP padding)

# https://pycryptodome.readthedocs.io/en/latest/src/examples.html
message = b'\x8f\x1c\xfc]\x08\xfe\x92\x0f\x1a\x8f\xe6L\xdb\x8d\x94\x0c\rR\x08$\xf8,s\x81\xf2\xe6\xbeF\xf6\xd1\xda\xe9]>\xb6u\xc6\xb4\x149Gs,\xa9\x10\xf1\xb6\xfc\xd3\x11\x17\xfc\x96\x7fr\xd2+\xf7\x9e\x9bq\x0c\xcb:z\xc3\xffi:\x98`\x9a\xab\x8c\x90\x15\xce1\xda\x07w\xabX,\x89\xeb\x17o\xb4f\xa7%\x9c\x0b\x1d8C\x98$\xe2\x95f+\xc7\x05\x11\xdb\x0ch\x9b\xe2\'\x12)\xf4 \xe5RZy_@)9\x95\xaf\xf9\xeb\xbao>t\x8d\xb8\xbc\x97NzFQ\n\xb2o\xbd\xdc\xea\xa4\xefBp\xc0\xf2Rj\x08\x9f\xa1\xcea\xd1\xa3\xe2\x96#n\x84\xdaV\x94&"\xf4q\xaa\xef\xb6R\xa1\xff\x9c\xe1\x82~2\xff;\xe3:\x7fM\xf6\x02NgQ\xf3pL)j\n\xcb\xa1\x04\xdc#\x87\xd6\xbb\x98\x9b\x1b\x87\x07\x11!)\xbeR\xfe\x18\xd1\xa5\xado\x1cm\xa2\xde7\xe8<\xd1\xc7\xa4OW/\xb6\xbfjy~y\x172\xf3N\xac\xf5\x89U$\x07\x83\xb0'

private_key = RSA.import_key(open("private_key.pem").read())
cipher_rsa = PKCS1_OAEP.new(private_key)
data = cipher_rsa.decrypt(message)

plaintext = data
print('plaintext:', plaintext)
# plaintext: b'You found your first flag!    cyber202{Gr3@tJ0b!}'

In [ ]:
########## Question 7 (30 points) ##########

# create your own RSA key, include the public file in your submission
# send me a message with your name, a message, and a class flag (like: cyber202{flag})
# digitally sign the encrypted message using PKCS#1 v1.5
# https://pycryptodome.readthedocs.io/en/latest/src/signature/pkcs1_v1_5.html
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes

message = b'cyber202{Korbin Coward: This a great class!}'

recipient_key = RSA.import_key(open("kc_public.pem").read())
session_key = get_random_bytes(16)

cipher_rsa = PKCS1_OAEP.new(recipient_key)
enc_session_key = cipher_rsa.encrypt(session_key)

cipher_aes = AES.new(session_key, AES.MODE_EAX)
ciphertext, tag = cipher_aes.encrypt_and_digest(message)

with open("encrypted_message.bin", "wb") as f:
    f.write(enc_session_key)
    f.write(cipher_aes.nonce)
    f.write(tag)
    f.write(ciphertext)

bundle = enc_session_key + cipher_aes.nonce + tag + ciphertext

key_private = RSA.import_key(open("kc_private_key.pem").read())
h = SHA256.new(bundle)
signature = pkcs1_15.new(key_private).sign(h)

with open("signature.bin","wb") as f:
    f.write(signature)
    
print('signature:', signature)